In [13]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import yfinance as yf

# List of stock tickers
tickers = ['BTC', 'ETH-USD', 'SOL']

In [14]:
# Download stock data
data = yf.download(tickers, start='2021-01-01', end='2022-12-31')['Adj Close']

[*********************100%***********************]  3 of 3 completed


In [15]:
# Calculate daily returns
returns = data.pct_change()

In [16]:
# Calculate the covariance matrix
covariance_matrix = returns.cov()

In [25]:
# Define the entropy function
def entropy(w, covariance_matrix):
    variances = np.diag(np.dot(w, covariance_matrix))
    return -np.sum(variances * np.log(variances + 1e-1000))

In [26]:
# Define the optimization constraints
constraints = [{'type': 'ineq', 'fun': lambda w: w}]

In [27]:
# Define the initial weights
initial_weights = [1/len(returns.columns) for i in range(len(returns.columns))]

In [28]:
bounds = [(0, 1) for i in range(len(initial_weights))]

In [29]:
result = minimize(entropy, initial_weights, args=(covariance_matrix,), bounds=bounds, constraints=constraints, method='SLSQP')

<ipython-input-25-b7938af124ad>:4: RuntimeWarning: divide by zero encountered in log
  return -np.sum(variances * np.log(variances + 1e-1000))
<ipython-input-25-b7938af124ad>:4: RuntimeWarning: invalid value encountered in multiply
  return -np.sum(variances * np.log(variances + 1e-1000))


In [30]:
optimal_weights = result.x

In [31]:
# Create a new DataFrame with the optimal weights and tickers
df = pd.DataFrame({'Ticker': tickers, 'Weight': optimal_weights})

# Calculate the percentage of each stock
df['Percentage'] = df['Weight'] * 100

# Display the DataFrame
print(df)

    Ticker    Weight  Percentage
0      BTC  0.333333   33.333333
1  ETH-USD  0.333333   33.333333
2      SOL  0.333333   33.333333


In [32]:
# Round the weights to 15 decimal places
optimal_weights_rounded = np.round(optimal_weights, 15)

# Create a new DataFrame with the rounded weights and the tickers as the index
results_df = pd.DataFrame(optimal_weights_rounded, index=tickers, columns=['Weight (%)'])

# Multiply the weights by 100 to express them as percentages
results_df['Weight (%)'] = results_df['Weight (%)']*100

# Print the DataFrame
print(results_df)

         Weight (%)
BTC       33.333333
ETH-USD   33.333333
SOL       33.333333
